# Activité 1 - Analysez vos données textuelles

NB : J'ai dû lancer jupyter avec la commande suivante :<br>
jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

Ceci pour éviter d'avoir l'erreur décrite ici: <br>
https://stackoverflow.com/questions/43288550/iopub-data-rate-exceeded-in-jupyter-notebook-when-viewing-image


### Objectif
Le but de cette activité est de nettoyer les données textuelles brutes fournies, et de créer un jeu de données d’entraînement en vue de créer un moteur de résumé automatique. 

Le jeu de données est téléchargeable ici: https://drive.google.com/uc?export=download&id=0BwmD_VLjROrfTHk4NFg2SndKcjQ


### Contexte
Les données brutes représentent un corpus d’articles CNN. L’objectif est de récupérer les features des documents et les highlights (résumés courts) associés concaténés, en vue d’entraîner un potentiel modèle de création de résumé d’articles.

### Consigne
Les opérations de traitement suivantes sont attendues sur le texte, pas forcément dans cet ordre :
* Créer des paires de document (article, highlights)
* Suppression de la ponctuation
* Séparation en token en minuscules
* Suppression des stopwords pour les articles
* Calcul des fréquences et tf-idf sur les deux types de documents
* Enregistrement du nouveau jeu de données d’entraînement pour usage ultérieur

### Etapes
#### 1. Lecture des données et création de paires (article, highlights).
* On passe le texte en minuscules dès cette étape.
   
   
#### 2. Tokenization des articles et des highlights.
* Les stopwords sont conservés pour les highlights, mais retirés pour les articles. 
* Les deux corpus tokenizés sont stockés dans des listes.    
* L'utilisation d'un RegexpTokenizer permet d'obtenir des tokens sans ponctuation.
   
   
#### 3. Calcul des fréquences et tf-idf sur les deux types de documents.
* Création de deux matrices de fréquences, une pour les articles et une pour les highlights, dénombrant le nombres d'occurrences de chaque mot dans chaque document.
* Création de deux matrices tf-idf, une pour les articles et une pour les highlights, stockant le tf-idf de chaque paire (document, mot). 
* Création de deux dictionnaires, un pour les articles et un pour les highlights, permettant de mapper les mots dénombrés dans les matrices ci-dessus à leur représentation sous forme d'entier. 
   
   
#### 4. Stockage des listes et dictionnaires créés dans un fichier pickle. 
* Stockage des listes tokenizées, matrices de fréquence et tf-idf dans un fichier pickle.
* On les recharge juste après pour vérifier que tout fonctionne. 


## I. Lecture des données et création de paires (article, highlights).
* Lecture du jeu d'entraînement.
* Création de deux listes, *articles* et *highlights*, contenant respectivement les articles et les résumés des fichiers du dataset passés en minuscules.
* Création d'une liste *pairs* contenant des tuples (article, highlight), constitués à partir des index de chaque texte dans les listes *articles* et *highlights*. 


In [2]:
import os

def load_dataset(directory, articles, highlights, pairs):
    """
    Browse the directory containing the dataset files.
    Load each file content in memory. 
    Replace all upper case letters with lower case letters.
    Add: 
      - The initial text in lower case to the articles list
      - The highlights to the highlights list
      - The pair of indexes to the pairs list
    """

    for entry in os.listdir(directory):
        
        with open(directory + '/' + entry, 'r', encoding="utf8") as file:
            
            observation = file.read().lower()
            splitted = observation.split("@highlight")
            articles.append(splitted[0])
            article_index = len(articles) - 1
            
            for highlight in splitted[1:]:
                highlights.append(highlight)
                highlight_index = len(highlights) - 1
                pairs.append((article_index, highlight_index))           
                    
articles = []
highlights = []
pairs = []
load_dataset('cnn/stories', articles, highlights, pairs)

## II. Tokenization des articles et des highlights.
* Les stopwords sont conservés pour les highlights, mais retirés pour les articles. 
* Les deux corpus tokenizés sont stockés dans des listes: *tokenized_articles* et *tokenized_highlights*    
* L'utilisation d'un RegexpTokenizer permet d'obtenir des tokens sans ponctuation.
   

In [4]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

stop_words = set(stopwords.words('english')) 
 
def preprocess(text, remove_stopwords=False):
    """
    Suppress all punctuation in the given text. 
    Remove the stopwords if remove_stopwords is True.
    """
    tokenizer = RegexpTokenizer(r'\w+')
    tokenized = tokenizer.tokenize(text)
    
    if remove_stopwords is True:
        tokenized = [w for w in tokenized if w not in stop_words]
    
    return tokenized

tokenized_articles = [preprocess(a, remove_stopwords=True) for a in articles]
tokenized_highlights = [preprocess(h) for h in highlights]

## III. Calcul des fréquences et tf-idf sur les deux types de documents.
* Création de deux matrices de fréquences, *counts_articles* et *counts_highlights*, dénombrant le nombres d'occurrences de chaque mot dans chaque document.
* Création de deux matrices tf-idf, *tfidf_articles* et *tfidf_highlights*, stockant le tf-idf de chaque paire (document, mot). 
* Création de deux dictionnaires, *vocabulary_articles* et *vocabulary_highlights*, permettant de mapper les mots dénombrés dans les matrices ci-dessus à leur représentation sous forme d'entier. 

In [5]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re

def find_frequency(text_list, vectorizer):
    """
    Apply the given vectorizer to the text_list.
    """
    # Remove punctuation.
    punctuation = re.compile('\W+')
    text_list = [punctuation.sub(r' ', text) for text in text_list]
    
    # Create a dictionary containing the count of each word of the vocabulary.
    word_count_vector = vectorizer.fit_transform(text_list)
    return word_count_vector

# Find the count of each word in the articles and highlights corpuses
cv_articles = CountVectorizer(stop_words=stop_words)
cv_highlights = CountVectorizer()

counts_articles = find_frequency(articles, cv_articles)
counts_highlights = find_frequency(highlights, cv_highlights)

# Find the tfidf for both the articles and highlights corpuses
tv_articles = TfidfVectorizer(stop_words=stop_words)
tv_highlights = TfidfVectorizer()

tfidf_articles = find_frequency(articles, tv_articles)
tfidf_highlights = find_frequency(highlights, tv_highlights)

# Find the vocabulary dictionaries for each corpus
vocabulary_articles = cv_articles.vocabulary_
vocabulary_highlights = cv_highlights.vocabulary_

## IV. Stockage des listes et dictionnaires créés dans un fichier pickle. 
* Stockage des listes tokenizées, matrices de fréquence et tf-idf dans un fichier pickle.
* On les recharge juste après pour vérifier que tout fonctionne. 
* On affiche un extrait de la matrice *counts_articles*, contenant, pour chaque couple (*document*, *word*), le nombre d'occurrences du mot *word* dans le document *document*.
* On affiche un extrait de la matrice *tfidf_highlights*, contenant, pour chaque couple (*document*, *word*), le tf-idf du mot *word* dans le document *document*.
* On vérifie sur un exemple que les paires (article, highlight) sont cohérentes.

In [6]:
import pickle

# Save the dataset to a pickle file
with open('data_activite1.pickle', 'wb') as pickle_file:
    
    pickle.dump(pairs, pickle_file)
    
    pickle.dump(tokenized_articles, pickle_file)
    pickle.dump(tokenized_highlights, pickle_file)
    
    pickle.dump(vocabulary_articles, pickle_file)
    pickle.dump(vocabulary_highlights, pickle_file)
    
    pickle.dump(counts_articles, pickle_file)
    pickle.dump(counts_highlights, pickle_file)
    
    pickle.dump(tfidf_articles, pickle_file)
    pickle.dump(tfidf_highlights, pickle_file)

In [7]:
# The dumped elements are loaded back into memory to check they were properly stored.
with open('data_activite1.pickle', 'rb') as pickle_file:
       
    pairs = pickle.load(pickle_file)
    
    tokenized_articles = pickle.load(pickle_file)
    tokenized_highlights = pickle.load(pickle_file)
    
    vocabulary_articles = pickle.load(pickle_file)
    vocabulary_highlights = pickle.load(pickle_file)
    
    counts_articles = pickle.load(pickle_file)
    counts_highlights = pickle.load(pickle_file)
    
    tfidf_articles = pickle.load(pickle_file)
    tfidf_highlights = pickle.load(pickle_file)


In [8]:
import random

print("\nSANITY CHECK 1")

# Sanity check of counts_articles.

number_articles = counts_articles.shape[0]
voc_size_articles = counts_articles.shape[1]

print("\nHere are 10 tuples (A, W) with the number of times the word W appears in the article A:")
m = '     The article {0:<6} contains {1:<3} time(s) the word {2:<8}'

for i in range(10):
    
    art = 0
    word = 0
    
    while counts_articles[art, word] == 0:
        art = random.randint(0, number_articles - 1)
        word = random.randint(0, voc_size_articles - 1)
        
    for w, i in vocabulary_articles.items():
        if i == word:
            break
    print(m.format(art, counts_articles[art, word],  w))


print("\nSANITY CHECK 2")

# Sanity check of tfidf_highlights.

number_highlights = tfidf_highlights.shape[0]
voc_size_highlights = tfidf_highlights.shape[1]

print("\nHere are 10 tuples (H, W) with the tfidf of the word W in the highlight H:")
m = '     The word {2:<15} in the highlight {0:<8} has a tf-idf of {1:<8}.'

for i in range(10):
    
    high = 0
    word = 0
    
    while tfidf_highlights[high, word] == 0:
        high = random.randint(0, number_highlights - 1)
        word = random.randint(0, voc_size_highlights - 1)
        
    for w, i in vocabulary_highlights.items():
        if i == word:
            break
    print(m.format(high, tfidf_highlights[high, word],  w))


print("\nSANITY CHECK 3")

# Sanity check of the (article, highlight) pairs.
pair = random.randint(0, len(pairs)-1)
print("\nThe following highlight sums up the following tokenized article:")
print("\nHighlight:")
print(" ".join(tokenized_highlights[pairs[pair][1]]))

print("\nTokenized article:")
print(" ".join(tokenized_articles[pairs[pair][0]]))


SANITY CHECK 1

Here are 10 tuples (A, W) with the number of times the word W appears in the article A:
     The article 39244  contains 4   time(s) the word ohio    
     The article 20728  contains 1   time(s) the word hour    
     The article 2074   contains 1   time(s) the word able    
     The article 61799  contains 7   time(s) the word prix    
     The article 67135  contains 1   time(s) the word oldest  
     The article 62691  contains 1   time(s) the word 20      
     The article 24711  contains 3   time(s) the word pun     
     The article 55075  contains 3   time(s) the word bush    
     The article 25631  contains 1   time(s) the word surface 
     The article 71987  contains 1   time(s) the word national

SANITY CHECK 2

Here are 10 tuples (H, W) with the tfidf of the word W in the highlight H:
     The word kill            in the highlight 142594   has a tf-idf of 0.25769197225374746.
     The word criticism       in the highlight 239344   has a tf-idf of 0.316936